#FastText: 내부 단어도 고려하는 word embedding

Word2Vec의 확장판이다. Word2Vec은 하나의 단어를 쪼갤 수 없는 단위로 생각하지만 FastText는 하나의 단어 내부를 여러 단어로 나눈다.

== 단어 하나를 n-gram을 통해 여러 subword으로 나눈다

n=3일 때, apple -> <ap, app, ppl, ple, le>로 나눈다. (< 와 >은 시작과 끝을 의미한다.)
 이 내부 subword들을 하나의 토큰으로 인식하여 벡터로 만든다. 또, 여기서 추가로 하나를 더 벡터화하는데, 기존 단어에 <와 >을 붙인 < apple >도 토큰으로 만든다.

 정리해서, apple이라는 단어를 벡터화할 때는, <ap + app + ppl + ple + le>를 더한 값을 이용한다.

#Word2Vec와 비교하였을 때 얻을 수 있는 강점 1.

Out of vocabulary에 대한 대응 성능이 좋아진다.

ex) Word2Vec는 birthplace라는 단어가 vocab에 없을 때, 이를 그냥 OOV(out of vocabulary)로 처리한다. 하지만 FastText는 birthplace를 birth + place라는 내부 단어로 나눌 수 있으므로 birthplace라는 단어가 vocab에 없어도 이를 처리할 수 있다.

#Word2Vec와 비교하였을 때 얻을 수 있는 강점 2.

Word2Vec은 등장 빈도 수가 적은 단어에 대해서 weight update를 자주 할 수 없기 때문에 임베딩의 정확도가 높지 않다. 하지만 FastText는 등장 빈도 수가 적은 단어의 subword가 등장 빈도 수가 많은 단어의 subword와 곂친다면, 임베딩을 비교적 더 좋게 할 수 있다. 이런 이유 때문에, 맞춤법이 잘 지켜지지 않은 corpus에 대해서는 fasttext가 성능이 더 좋다.


#Word2Vec와 FastText 비교

In [11]:
pip install nltk

In [12]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")


('ted_en-20160408.xml', <http.client.HTTPMessage at 0x79b603648b50>)

In [13]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0)



In [14]:
model.wv.most_similar("electrofishing")

KeyError: "Key 'electrofishing' not present in vocabulary"

In [17]:
from gensim.models import FastText

model = FastText(result, vector_size=100, window=5, min_count=5, workers=4, sg=1)


In [18]:
model.wv.most_similar("electrofishing")


[('electrolux', 0.8728017807006836),
 ('electrolyte', 0.8689448833465576),
 ('electro', 0.8544501066207886),
 ('electroshock', 0.851844072341919),
 ('electroencephalogram', 0.8349860906600952),
 ('electronic', 0.8321779370307922),
 ('electrogram', 0.8268095850944519),
 ('electrochemical', 0.8254574537277222),
 ('electron', 0.8214612007141113),
 ('electric', 0.8171874284744263)]